In [1]:
import sys
sys.path.append('../..')

import jieba
import torch
import pickle
import nlp_library
import transformers 
import torch.nn as nn
import torch.optim as optim

from transformers import AutoTokenizer
from transformers import BertConfig
from nlp_library.dataset import NERDataset
from nlp_library.processor.vocab import CharVocab
from nlp_library.processor.tokenizer.ner import TransfomerTokenizer
from nlp_library.neural_network import VanillaBert
from nlp_library.factory.task import NamedentityTask

### 一、数据读入与处理

#### 1. 数据读入

In [2]:
ner_train_dataset = NERDataset('../data/task_datasets/hello_work/train_data.csv')
ner_dev_dataset = NERDataset('../data/task_datasets/hello_work/dev_data.csv', categories=ner_train_dataset.categories)

#### 2. 词典创建

In [3]:
import transformers 
from transformers import AutoTokenizer

bert_vocab = transformers.AutoTokenizer.from_pretrained('nghuyong/ernie-1.0')

#### 3. 生成分词器

In [4]:
max_seq_length = 150

tokenizer = TransfomerTokenizer(max_seq_length, bert_vocab)

#### 4. ID化

In [5]:
ner_train_dataset.convert_to_ids(tokenizer)
ner_dev_dataset.convert_to_ids(tokenizer)

<br>

### 二、模型构建

#### 1. 模型参数设置

In [6]:
from transformers import BertConfig

bert_config = BertConfig.from_pretrained('nghuyong/ernie-1.0', 
                                         num_labels=len(ner_train_dataset.cat2id))

#### 2. 模型创建

In [7]:
torch.cuda.empty_cache()

In [13]:
dl_module = VanillaBert.from_pretrained('../pretrained_model/nghuyong/ernie-1.0/', 
                                        config=bert_config)

Some weights of the model checkpoint at ../pretrained_model/nghuyong/ernie-1.0/ were not used when initializing VanillaBert: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing VanillaBert from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing VanillaBert from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of VanillaBert were not initialized from the model checkpoint at ../pretrained_model/nghuyong/ernie-1.0/ and are newly initialized: ['classifi

<br>

### 三、任务构建

#### 1. 任务参数和必要部件设定

In [14]:
# 设置运行次数
num_epoches = 10
batch_size = 64

In [15]:
param_optimizer = list(dl_module.named_parameters())
param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]      

#### 2. 任务创建

In [16]:
model = NamedentityTask(dl_module, 'adamw', 'ce')

#### 3. 训练

In [ ]:
model.fit(ner_train_dataset, 
          ner_dev_dataset,
          lr=1e-4,
          epochs=num_epoches, 
          batch_size=batch_size,
          params=optimizer_grouped_parameters
         )